# Foundations of CNN

* used mostly in computer-vision problems such as image classification, object detection, style transfer, others
* images may lead to large inputs (and consequently very large networks)

## Convolution operation

**Convolution**  
* image (6 x 6) * filter (3 x 3) = result (4 x 4)
    * (n x n) * (f x f) = (n-f+1 x n-f+1)
        * f usually odd
        * not full detection, image shrinks -> padding (adding pixels around)
        * if p denotes padding size then the result size is (n+2p-f+1 x n+2p-f+1)
        * valid convolution (no padding)
            * (n x n) * (f x f) = (n-f+1 x n-f+1)
        * same convolution (output size is same as original input size)
            * (n+p x n+p) * (f x f) = (n+2p-f+1 x n+2p-f+1)
            * p = (f-1)/2  
* in some textbooks, filters are rotated (mirrored) before doing the transformation (to allow for associations), by convention this is not used in deep learning literature
* filters can be learned through weights (!)        
    
Algorithm 
* sliding filter window into the image, multiplying filter with the image values, adding them together and putting them into the result matrix
* sliding through the whole image from left to right (by 1 pixel), from top to down (by 1 pixel)
* result matrix is filled, serves as filter detection (could be ie edge filter)
* python: `conv-forward` tensorflow: `tf.nn.conv2d` keras: `tf.kearas.Conv2D`

**Strides**
* strides (movement by pixels) different from 1 can be used for the filter window
* (n+p x n+p) * (f x f) = ([(n+2p-f)/s+1] x [(n+2p-f)/s+1])
    * s is stride, if the fraction is not integer we round down

**Volumes**
* image (6 x 6 x 3) * filter (3 x 3 x 3) = result (4 x 4)  
* detection across color channels (can look at one or all)
* multiple filters can be applied at the same time resulting in stacking the 2d outputs (the last dimension of the result is driven by number of convolution filters)

Algorithm
* put filter into the starting position, do element-wise multiplication for     corresponding elements
* sum the result and add it to the result matrix
* move the filter by stride and repeat until the result matrix is filled

## One layer of CNN

* number of params -> (filter size + bias) * number of filters
* notation for layer $l$
    * $m$ -> training examples
    * $f^{[l]}$ -> filter size
    * $p^{[l]}$ -> padding
    * $d^{[l]}$ -> stride
    * $n_c^{[l]}$ -> number of filters
    * $f^{[l]}$ x $f^{[l]}$ x $n_c^{[l-1]}$ -> size of a filter, last dimension same as the number of channels in previous layer
    * $f^{[l]}$ x $f^{[l]}$ x $n_c^{[l]}$ -> activations
    * $m$ x $f^{[l]}$ x $f^{[l]}$ x $n_c^{[l]}$-> vectorized activations
    * $f^{[l]}$ x $f^{[l]}$ x $n_c^{[l-1]}$ x $n_c^{[l]}$ -> weights
    * $n_c^{[l]}$ -> bias

* input -> $n_h^{[l-1]}$ x $n_w^{[l-1]}$ x $n_c^{[l-1]}$  
* output -> $n_h^{[l]}$ x $n_w^{[l]}$ x $n_c^{[l]}$, where $n_h^{[k]}= [\frac{n_w^{[l-1]}+2p^{[l]}-f^{[l]}}{s^{[l]}}+1]$, see less formally written version above

* forward step (one filter)
    * volume calculation (sliding, element-wise multiplication and sum)
    * adding bias
    * feeding the resulting matrix into an activation function
    * stacking filters together (output of the convolution layer)
    * ([(n+2p-f)/s+1] x [(n+2p-f)/s+1] x number of filters)

* backward step ?


## Deep CNN

* for the parameters of convolution filters, formulas above apply
* common architecture -> convolution filter shrink and number of channels increases
* in the last step, filters are unrolled to one long vector which is fed into output layer

**Pooling layer**  
* max pooling -> input matrix divided into regions, returns max for each region
    * filter size & stride as pooling hyper params (defines regions)
    * no parameters to learn (fixed function)
    * intuition -> in case filter feature is detected, keep large number (the feature was detected)
    * done independently on each of the channel
    * ([(n+2p-f)/s+1] x [(n+2p-f)/s+1] x number of channels)
    
* average pooling -> input matrix divided into regions, returns avg for each region
    * not used that often (unless in very deep NNs for reducing the input)

## Example

* written digit detection problem 
* input layer
    * rgb image
    * (32 x 32 x 3)
* first layer
    * first step
        * CNN1
        * filter (5 x 5) , stride 1, no padding
        * 6 filters
        * (28 x 28 x 6)
    * second step
        * Pool1
        * max-pooling operation
        * filter (2 x 2), stride 2, no padding
        * (14 x 14 x 6)
    * first layer -> by convention based on learnable params

* second layer
    * first step
        * CNN2
        * filter (5 x 5), stride 1, no padding
        * (10 x 10 x 16)
    * second step
        * Pool2
        * max-pooling operation
        * filter (2 x 2), stride 2
        * (5 x 5 x 16)
        * flatten to 400 units

* third layer 
    * FC3 (fully connected)
    * 120 relu units
    * $W^{[3]} (120, 400)$

* forth layer
    * FC4 (fully connected)
    * 84 relu units

* output layer
    * 10 softmax units

| | Activation shape | Activation size (gradually decreases) | # parameters |
| - | - | - | - |
| Input | (32, 32, 3) | 3,072 | 0 |
| CNN1 (f=5, s=1) | (28, 28, 6) | 4,704 | 456 |
| Pool1 | (14, 14, 6) | 1,176| 0 |
| CNN2 (f=5, s=1)| (10, 10, 16) | 1,600 | 2,416 |
| Pool2 | (5, 5, 16) | 400| 0 |
| FC3 | (120, 1) | 120 | 48,120 (connection tu every unit from previous layer + bias -> 120*(400+1))|
| FC4 | (84, 1) | 84 | 10,164 (connection to every unit from previous layer + bias -> 84*(120+1))|

## Why convolutions

* parameter sharing and sparse connections
* imagine network from the previous example
    * input layer
        * rgb image
        * (32 x 32 x 3)
        * 3,072 activations
    * first layer
        * first step
            * CNN1
            * filter (5 x 5) , stride 1, no padding
            * 6 filters
            * (28 x 28 x 6)
            * 4,704 activations
        * second step
            * Pool1
            * max-pooling operation
            * filter (2 x 2), stride 2, no padding
            * (14 x 14 x 6)
    * for fully connected network, this would mean $3,072\ *\ 4,704 \sim 1.4 * 10^7$ parameters
    * for convolution network, this means $(5\ *\ 5\ *3 + 1) * 6$ params per filter, thus 456 params in total, this is based on equation $(f^{[l]}\cdot f^{[l]}\cdot n_c^{[l-1]}+1) \cdot n_c^{[l]}$
    
    Parameter sharing  
    * filters are reused across whole image (valid for both low and high-level features), in other words filter parameters are shared
    
    Sparsity of connections  
    * in each layer, output depends only on small number of inputs (sliding window region), in other words connections between layers are 
    
**Backward steps**
* loss & cost analogous to traditional MLP
* unfortunately no details were shared